# 📈 Metrics Analysis Notebook

This notebook analyzes **CPU** and **Memory** resource usage during performance experiments using Fortio and a service mesh (Istio or Linkerd). It reads resource usage data exported in CSV format from the Kubernetes metrics API and generates visualizations per container.

## 🧪 Experiments Covered

1. **01 - HTTP Max Throughput**
    - Measures max throughput with default Fortio settings.
    - Generates average CPU and memory usage per container.

2. **02 - HTTP Constant Throughput**
    - Uses constant QPS values: 1, 1000, and 10000.
    - Shows how resource usage changes with increasing load.

3. **03 - HTTP Payload Variation**
    - Fixed QPS (100) with payload sizes: 0, 1000, and 10000 bytes.
    - Compares container resource usage with different payload sizes.

## 📁 Expected Input

- CSV files under `../results/<experiment>/metrics_<mesh>_<qps>_<payload>_<timestamp>.csv`
- Structure:
    - `timestamp, namespace, pod, container, cpu(n), memory(Ki)`
    - Plus derived fields: `cpu` (as float), `memory` (as float)

## 📉 Output

- CPU and Memory bar charts per container for each experiment setup.
- Output PNGs are saved in:
    - `../diagrams/<experiment>/cpu_*.png`
    - `../diagrams/<experiment>/memory_*.png`

> 📌 **Note**: CPU values are shown in nanocores, and memory in Ki.

In [3]:
# %% [code] Import required libraries
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

# Use a default matplotlib style
plt.style.use('default')

In [5]:
def convert_cpu(cpu_str):
    """
    Convert CPU usage strings to millicores (m).

    Conversions:
      - '123n' → nanocores → 0.000123 m
      - '123u' → microcores → 0.123 m
      - '123m' → millicores → 123.0 m
      - '123'  → cores     → 123000.0 m
    """
    try:
        s = cpu_str.lower().strip()
        if s.endswith('n'):
            value = float(s[:-1])
            return value / 1e6
        elif s.endswith('u'):
            value = float(s[:-1])
            return value / 1e3
        elif s.endswith('m'):
            return float(s[:-1])
        else:
            value = float(s)
            return value * 1000.0
    except Exception as e:
        print(f"Error converting CPU value '{cpu_str}': {e}")
        return None

def convert_memory(mem_str):
    """
    Convert memory usage from a string (which might be in Ki, Mi, or Gi) to a float representing megabytes (MB).
    
    Conversions:
    - If the value ends with "Ki": MB = (numeric value) / 1024.
    - If the value ends with "Mi": MB = numeric value.
    - If the value ends with "Gi": MB = (numeric value) * 1024.
    - If no known suffix, attempt direct conversion.
    """
    try:
        if mem_str.endswith("Ki"):
            value = float(mem_str[:-2])
            mb = value / 1024.0
            return mb
        elif mem_str.endswith("Mi"):
            value = float(mem_str[:-2])
            # Assuming 1 MiB is reported as 1 MB
            return value
        elif mem_str.endswith("Gi"):
            value = float(mem_str[:-2])
            return value * 1024.0
        else:
            # If no unit present, try converting directly
            return float(mem_str)
    except Exception as e:
        print(f"Error converting memory value '{mem_str}': {e}")
        return None


In [6]:
def load_metrics_csv(file_path):
    """
    Reads a CSV file with resource metrics, parses the timestamp, and converts the raw CPU
    (nanocores remain unchanged) and memory values (in Ki).
    
    Returns:
        A pandas DataFrame with columns:
        - timestamp
        - namespace
        - pod
        - container
        - cpu(n) (original)
        - memory(Ki) (original)
        - cpu: numeric CPU in millicore
        - memory: numeric memory in MB
    """
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['cpu'] = df['cpu(n)'].apply(convert_cpu)
    df['memory'] = df['memory(Ki)'].apply(convert_memory)
    return df

def extract_mesh_qps_payload(filename):
    """
    Extract mesh, qps, and payload from a filename assumed to be formatted as:
      metrics_<mesh>_<qps>_<payload>_<timestamp>.csv
    Returns a tuple: (mesh (str), qps (int), payload (int))
    """
    base = os.path.basename(filename)
    parts = base.replace("metrics_", "").replace(".csv", "").split("_")
    if len(parts) < 3:
        return None, None, None
    mesh, qps, payload = parts[0], parts[1], parts[2]
    return mesh, int(qps), int(payload)

def shorten_label(ns, pod, container, max_len=18):
    """
    Shortens the namespace, pod, and container names to a maximum length.
    If the name exceeds max_len, it truncates the string and appends '..'.
    Returns a formatted string: "ns/pod/container".
    """
    ns_abbr = ns if len(ns) <= max_len else ns[:max_len - 2] + '..'
    pod_abbr = pod if len(pod) <= max_len else pod[:max_len - 2] + '..'
    container_abbr = container if len(container) <= max_len else container[:max_len - 2] + '..'
    return f"{ns_abbr}/{pod_abbr}/{container_abbr}"

In [8]:
# HTTP Max throughput experiment (experiment 1).
experiment_pattern = os.path.join("..", "results", "*", "01_http_max_throughput", "metrics_*.csv")
output_dir         = os.path.join("..", "diagrams", "01_http_max_throughput")
csv_files          = glob.glob(experiment_pattern)
os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", experiment_pattern)
else:
    df_list   = [load_metrics_csv(file) for file in csv_files]
    df_exp    = pd.concat(df_list, ignore_index=True)
    df_exp    = df_exp.dropna(subset=['cpu', 'memory'])
    df_cp     = df_exp[df_exp['group'] == 'control-plane'].copy()
    df_dp     = df_exp[df_exp['group'] == 'data-plane'].copy()
    agg_stats = df_exp.groupby(['namespace', 'pod', 'container'])[['cpu', 'memory']].agg(['mean', 'median', 'std'])

    for metric, ylabel in [('cpu', "CPU Usage (millicore)"), ('memory', "Memory Usage (MB)")]:
        metric_title = "CPU" if metric == "cpu" else "Memory"

        # --- Global Max Plot (aggregated over all repetitions) ---
        max_global = df_exp.groupby(['namespace', 'pod', 'container'])[metric].max()
        labels_global = [shorten_label(ns, pod, c) for ns, pod, c in max_global.index]
        max_global.index = labels_global    
        plt.figure(figsize=(15, 10))
        max_global.plot(kind='bar', color='tab:orange')
        plt.title(f"Experiment 1: Max Throughput\n(Prot=HTTP, Metric={metric_title})")
        plt.ylabel(ylabel)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"metrics_{metric}_0.png"))
        plt.close()

        # --- Control Plane Stacked Plot ---
        agg_cp = df_cp.groupby(['namespace', 'pod', 'container'])[metric].max().reset_index()
        cp_pivot = agg_cp.pivot_table(index=['namespace', 'pod'], columns='container', values=metric, fill_value=0)
        plt.figure(figsize=(15, 10))
        cp_pivot.plot(kind='bar', stacked=True)
        plt.title(f"Experiment 1: Max Throughput\n(Prot=HTTP, Metric={metric_title}), Control Plane")
        plt.ylabel(ylabel)
        plt.legend(loc='best', fontsize='small', ncol=4)
        plt.xticks(rotation=45, ha='right', fontsize=6)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"metrics_control_plane_{metric}_0.png"))
        plt.close()

        # --- Data Plane Plot ---
        max_dp = df_dp.groupby(['namespace', 'pod', 'container'])[metric].max()
        labels_dp = [shorten_label(ns, pod, c) for ns, pod, c in max_dp.index]
        max_dp.index = labels_dp
        plt.figure(figsize=(15, 10))
        max_dp.plot(kind='bar', color='tab:orange')
        plt.title(f"Experiment 1: Max Throughput\n(Prot=HTTP, Metric={metric_title}), Data Plane")
        plt.ylabel(ylabel)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"metrics_data_plane_{metric}_0.png"))
        plt.close()

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

In [9]:
# gRPC Max throughput experiment (experiment 2).
experiment_pattern = os.path.join("..", "results", "*", "01_http_max_throughput", "metrics_*.csv")
output_dir         = os.path.join("..", "diagrams", "01_http_max_throughput")
csv_files          = glob.glob(experiment_pattern)
os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", experiment_pattern)
else:
    # load all repetitions' CSVs and concatenate
    df_list = [load_metrics_csv(file) for file in csv_files]
    df_exp  = pd.concat(df_list, ignore_index=True)
    df_exp  = df_exp.dropna(subset=['cpu', 'memory'])
    df_cp   = df_exp[df_exp['group'] == 'control-plane'].copy()
    df_dp   = df_exp[df_exp['group'] == 'data-plane'].copy()
    agg_stats = df_exp.groupby(['namespace', 'pod', 'container'])[['cpu', 'memory']].agg(['mean', 'median', 'std'])

    for metric, ylabel in [('cpu', "CPU Usage (millicore)"), ('memory', "Memory Usage (MB)")]:
        metric_title = "CPU" if metric == "cpu" else "Memory"

        # --- Global Max Plot (aggregated over all repetitions) ---
        max_global = df_exp.groupby(['namespace', 'pod', 'container'])[metric].max()
        labels_global = [shorten_label(ns, pod, c) for ns, pod, c in max_global.index]
        max_global.index = labels_global    
        plt.figure(figsize=(15, 10))
        max_global.plot(kind='bar', color='tab:orange')
        plt.title(f"Experiment 1: Max Throughput\n(Prot=HTTP, Metric={metric_title})")
        plt.ylabel(ylabel)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"metrics_{metric}_0.png"))
        plt.close()

        # --- Control Plane Stacked Plot ---
        agg_cp = df_cp.groupby(['namespace', 'pod', 'container'])[metric].max().reset_index()
        cp_pivot = agg_cp.pivot_table(index=['namespace', 'pod'], columns='container', values=metric, fill_value=0)
        plt.figure(figsize=(15, 10))
        cp_pivot.plot(kind='bar', stacked=True)
        plt.title(f"Experiment 1: Max Throughput\n(Prot=HTTP, Metric={metric_title}), Control Plane")
        plt.ylabel(ylabel)
        plt.legend(loc='best', fontsize='small', ncol=4)
        plt.xticks(rotation=45, ha='right', fontsize=6)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"metrics_control_plane_{metric}_0.png"))
        plt.close()

        # --- Data Plane Plot ---
        max_dp = df_dp.groupby(['namespace', 'pod', 'container'])[metric].max()
        labels_dp = [shorten_label(ns, pod, c) for ns, pod, c in max_dp.index]
        max_dp.index = labels_dp
        plt.figure(figsize=(15, 10))
        max_dp.plot(kind='bar', color='tab:orange')
        plt.title(f"Experiment 1: Max Throughput\n(Prot=HTTP, Metric={metric_title}), Data Plane")
        plt.ylabel(ylabel)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"metrics_data_plane_{metric}_0.png"))
        plt.close()

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

In [10]:
# HTTP Constant throughput experiment (experiment 3)
results_pattern = os.path.join("..", "results", "*", "03_http_constant_throughput", "metrics_*.csv")
output_dir      = os.path.join("..", "diagrams", "03_http_constant_throughput")
csv_files       = glob.glob(results_pattern)
os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", results_pattern)
else:
    df_list = []
    for file in csv_files:
        df = load_metrics_csv(file)
        _, qps, _ = extract_mesh_qps_payload(file)
        df['qps'] = qps
        df_list.append(df)
    df_exp = pd.concat(df_list, ignore_index=True)
    df_exp = df_exp.dropna(subset=['cpu', 'memory', 'qps'])
    unique_qps = sorted(df_exp['qps'].unique())

    for qps_val in unique_qps:
        df_qps    = df_exp[df_exp['qps'] == qps_val]
        df_qps_cp = df_qps[df_qps['group'] == 'control-plane']
        df_qps_dp = df_qps[df_qps['group'] == 'data-plane']

        for metric, ylabel in [('cpu', "CPU Usage (millicore)"), ('memory', "Memory Usage (MB)")]:
            metric_title = "CPU" if metric == "cpu" else "Memory"

            # --- Global Plot ---
            max_global = df_qps.groupby(['namespace', 'pod', 'container'])[metric].max()
            labels_global = [shorten_label(ns, pod, c) for ns, pod, c in max_global.index]
            max_global.index = labels_global    
            plt.figure(figsize=(15, 10))
            max_global.plot(kind='bar', color='tab:orange')
            plt.title(f"Experiment 3: Constant Throughput\n(Prot=HTTP, QPS={qps_val}, Metric={metric_title})")
            plt.ylabel(ylabel)
            plt.xlabel("Container")
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"metrics_{metric}_{qps_val}.png"))
            plt.close()

            # --- Control Plane Stacked Plot ---
            agg_cp = df_qps_cp.groupby(['namespace', 'pod', 'container'])[metric].max().reset_index()
            cp_pivot = agg_cp.pivot_table(index=['namespace', 'pod'], columns='container', values=metric, fill_value=0)
            plt.figure(figsize=(15, 10))
            cp_pivot.plot(kind='bar', stacked=True)
            plt.title(f"Experiment 3: Constant Throughput\n(Prot=HTTP, QPS={qps_val}, Metric={metric_title}, Control Plane)")
            plt.ylabel(ylabel)
            plt.xlabel("Container")
            plt.legend(loc='best', fontsize='small', ncol=4)
            plt.xticks(rotation=45, ha='right', fontsize=6)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"metrics_control_plane_{metric}_{qps_val}.png"))
            plt.close()

            # --- Data Plane Plot ---
            max_dp = df_qps_dp.groupby(['namespace', 'pod', 'container'])[metric].max()
            labels_dp = [shorten_label(ns, pod, c) for ns, pod, c in max_dp.index]
            max_dp.index = labels_dp
            plt.figure(figsize=(15, 10))
            max_dp.plot(kind='bar', color='tab:orange')
            plt.title(f"Experiment 3: Constant Throughput\n(Prot=HTTP, QPS={qps_val}, Metric={metric_title}, Data Plane)")
            plt.ylabel(ylabel)
            plt.xlabel("Container")
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"metrics_data_plane_{metric}_{qps_val}.png"))
            plt.close()

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

In [11]:
# gRPC Constant throughput experiment (experiment 4)
results_pattern = os.path.join("..", "results", "*", "04_grpc_constant_throughput", "metrics_*.csv")
output_dir      = os.path.join("..", "diagrams", "04_grpc_constant_throughput")
csv_files       = glob.glob(results_pattern)
os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", results_pattern)
else:
    df_list = []
    for file in csv_files:
        df = load_metrics_csv(file)
        _, qps, _ = extract_mesh_qps_payload(file)
        df['qps'] = qps
        df_list.append(df)
    df_exp = pd.concat(df_list, ignore_index=True)
    df_exp = df_exp.dropna(subset=['cpu', 'memory', 'qps'])
    unique_qps = sorted(df_exp['qps'].unique())
    for qps_val in unique_qps:
        df_qps    = df_exp[df_exp['qps'] == qps_val]
        df_qps_cp = df_qps[df_qps['group'] == 'control-plane']
        df_qps_dp = df_qps[df_qps['group'] == 'data-plane']

        for metric, ylabel in [('cpu', "CPU Usage (millicore)"), ('memory', "Memory Usage (MB)")]:
            metric_title = "CPU" if metric == "cpu" else "Memory"

            # --- Global Plot ---
            max_global = df_qps.groupby(['namespace', 'pod', 'container'])[metric].max()
            labels_global = [shorten_label(ns, pod, c) for ns, pod, c in max_global.index]
            max_global.index = labels_global    
            plt.figure(figsize=(15, 10))
            max_global.plot(kind='bar', color='tab:orange')
            plt.title(f"Experiment 4: Constant Throughput\n(Prot=gRPC, QPS={qps_val}, Metric={metric_title})")
            plt.ylabel(ylabel)
            plt.xlabel("Container")
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"metrics_{metric}_{qps_val}.png"))
            plt.close()

            # --- Control Plane Stacked Plot ---
            agg_cp = df_qps_cp.groupby(['namespace', 'pod', 'container'])[metric].max().reset_index()
            cp_pivot = agg_cp.pivot_table(index=['namespace', 'pod'], columns='container', values=metric, fill_value=0)
            plt.figure(figsize=(15, 10))
            cp_pivot.plot(kind='bar', stacked=True)
            plt.title(f"Experiment 4: Constant Throughput\n(Prot=gRPC, QPS={qps_val}, Metric={metric_title}, Control Plane)")
            plt.ylabel(ylabel)
            plt.xlabel("Container")
            plt.legend(loc='best', fontsize='small', ncol=4)
            plt.xticks(rotation=45, ha='right', fontsize=6)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"metrics_control_plane_{metric}_{qps_val}.png"))
            plt.close()
            
            # --- Data Plane Plot ---
            max_dp = df_qps_dp.groupby(['namespace', 'pod', 'container'])[metric].max()
            labels_dp = [shorten_label(ns, pod, c) for ns, pod, c in max_dp.index]
            max_dp.index = labels_dp
            plt.figure(figsize=(15, 10))
            max_dp.plot(kind='bar', color='tab:orange')
            plt.title(f"Experiment 4: Constant Throughput\n(Prot=gRPC, QPS={qps_val}, Metric={metric_title}, Data Plane)")
            plt.ylabel(ylabel)
            plt.xlabel("Container")
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"metrics_data_plane_{metric}_{qps_val}.png"))
            plt.close()


<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

In [12]:
# HTTP Constant throughput with Payload experiment (experiment 5)
results_pattern = os.path.join("..", "results", "*", "05_http_payload", "metrics_*.csv")
output_dir      = os.path.join("..", "diagrams", "05_http_payload")
csv_files       = glob.glob(results_pattern)
os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", results_pattern)
else:
    df_list = []
    for file in csv_files:
        df = load_metrics_csv(file)
        mesh, qps, payload = extract_mesh_qps_payload(file)
        df['mesh']    = mesh
        df['qps']     = qps
        df['payload'] = payload
        df_list.append(df)
    df_exp = pd.concat(df_list, ignore_index=True)

    # Identify unique QPS and payload values
    unique_qps     = sorted(df_exp['qps'].dropna().unique())
    unique_payload = sorted(df_exp['payload'].dropna().unique())
    qps_val        = unique_qps[0] if unique_qps else None

    for payload_val in unique_payload:
        df_subset   = df_exp[df_exp['payload'] == payload_val]
        df_subset_cp = df_subset[df_subset['group'] == 'control-plane']
        df_subset_dp = df_subset[df_subset['group'] == 'data-plane']

        for metric, ylabel in [('cpu', "CPU Usage (millicore)"), ('memory', "Memory Usage (MB)")]:
            metric_title = "CPU" if metric == "cpu" else "Memory"

            # --- Global Plot ---
            max_global = df_subset.groupby(['namespace', 'pod', 'container'])[metric].max()
            labels_global = [shorten_label(ns, pod, c) for ns, pod, c in max_global.index]
            max_global.index = labels_global    
            plt.figure(figsize=(15, 10))
            max_global.plot(kind='bar', color='tab:orange')
            plt.title(f"Experiment 5: Constant Throughput\n(Prot=HTTP, QPS={qps_val}, Payload={payload_val}, Metric={metric_title})")
            plt.ylabel(ylabel)
            plt.xlabel("Container")
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"metrics_{metric}_{qps_val}_{payload_val}.png"))
            plt.close()

            # --- Control Plane Stacked Plot ---
            agg_cp = df_subset_cp.groupby(['namespace', 'pod', 'container'])[metric].max().reset_index()
            cp_pivot = agg_cp.pivot_table(index=['namespace', 'pod'], columns='container', values=metric, fill_value=0)
            plt.figure(figsize=(15, 10))
            cp_pivot.plot(kind='bar', stacked=True)
            plt.title(f"Experiment 5: Constant Throughput\n(Prot=HTTP, QPS={qps_val}, Payload={payload_val}, Metric={metric_title}, Control Plane)")
            plt.ylabel(ylabel)
            plt.xlabel("Container")
            plt.legend(loc='best', fontsize='small', ncol=4)
            plt.xticks(rotation=45, ha='right', fontsize=6)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"metrics_control_plane_{metric}_{qps_val}_{payload_val}.png"))
            plt.close()

            # --- Data Plane Plot ---
            max_dp = df_subset_dp.groupby(['namespace', 'pod', 'container'])[metric].max()
            labels_dp = [shorten_label(ns, pod, c) for ns, pod, c in max_dp.index]
            max_dp.index = labels_dp
            plt.figure(figsize=(15, 10))
            max_dp.plot(kind='bar', color='tab:orange')
            plt.title(f"Experiment 5: Constant Throughput\n(Prot=HTTP, QPS={qps_val}, Payload={payload_val}, Metric={metric_title}, Data Plane)")
            plt.ylabel(ylabel)
            plt.xlabel("Container")
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"metrics_data_plane_{metric}_{qps_val}_{payload_val}.png"))
            plt.close()

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

In [13]:
# HTTP Constant throughput with HTTPRoute header-based routing experiment (experiment 6)
results_pattern = os.path.join("..", "results", "*", "06_http_constant_throughput_header", "metrics_*.csv")
output_dir      = os.path.join("..", "diagrams", "06_http_constant_throughput_header")
csv_files       = glob.glob(results_pattern)
os.makedirs(output_dir, exist_ok=True)

if not csv_files:
    print("No CSV files found in:", results_pattern)
else:
    df_list = []
    for file in csv_files:
        df = load_metrics_csv(file)
        _, qps, _ = extract_mesh_qps_payload(file)
        df['qps'] = qps
        df_list.append(df)
    df_exp = pd.concat(df_list, ignore_index=True)
    df_exp = df_exp.dropna(subset=['cpu', 'memory', 'qps'])
    unique_qps = sorted(df_exp['qps'].unique())

    for qps_val in unique_qps:
        df_qps    = df_exp[df_exp['qps'] == qps_val]
        df_qps_cp = df_qps[df_qps['group'] == 'control-plane']
        df_qps_dp = df_qps[df_qps['group'] == 'data-plane']

        for metric, ylabel in [('cpu', "CPU Usage (millicore)"), ('memory', "Memory Usage (MB)")]:
            metric_title = "CPU" if metric == "cpu" else "Memory"

            # --- Global Plot ---
            max_global = df_qps.groupby(['namespace', 'pod', 'container'])[metric].max()
            labels_global = [shorten_label(ns, pod, c) for ns, pod, c in max_global.index]
            max_global.index = labels_global
            plt.figure(figsize=(15, 10))
            max_global.plot(kind='bar', color='tab:orange')
            plt.title(f"Experiment 6: Constant Throughput\n(Prot=HTTPRoute header, QPS={qps_val}, Metric={metric_title})")
            plt.ylabel(ylabel)
            plt.xlabel("Container")
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"metrics_{metric}_{qps_val}.png"))
            plt.close()

            # --- Control Plane Stacked Plot ---
            agg_cp = df_qps_cp.groupby(['namespace', 'pod', 'container'])[metric].max().reset_index()
            cp_pivot = agg_cp.pivot_table(index=['namespace', 'pod'], columns='container', values=metric, fill_value=0)
            plt.figure(figsize=(15, 10))
            cp_pivot.plot(kind='bar', stacked=True)
            plt.title(f"Experiment 6: Constant Throughput\n(Prot=HTTPRoute header, QPS={qps_val}, Metric={metric_title}, Control Plane)")
            plt.ylabel(ylabel)
            plt.xlabel("Container")
            plt.legend(loc='best', fontsize='small', ncol=4)
            plt.xticks(rotation=45, ha='right', fontsize=6)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"metrics_control_plane_{metric}_{qps_val}.png"))
            plt.close()

            # --- Data Plane Plot ---
            max_dp = df_qps_dp.groupby(['namespace', 'pod', 'container'])[metric].max()
            labels_dp = [shorten_label(ns, pod, c) for ns, pod, c in max_dp.index]
            max_dp.index = labels_dp
            plt.figure(figsize=(15, 10))
            max_dp.plot(kind='bar', color='tab:orange')
            plt.title(f"Experiment 6: Constant Throughput\n(Prot=HTTPRoute header, QPS={qps_val}, Metric={metric_title}, Data Plane)")
            plt.ylabel(ylabel)
            plt.xlabel("Container")
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"metrics_data_plane_{metric}_{qps_val}.png"))
            plt.close()

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>